***Readme***

This code helps calculate the current value for a meta goal based on how far above the road are latest datapoints across a list of goals.

 Before using, mark the list of the goals which you need to take into account with a unique tag. (It only makes sense to take into account some goals and not others. For example, it doesn't work for do-less goals. Works best for goals where the direction of data accumulation is monotonous. E.g. odometer or do-more goals where the total can only become higher and whittle-down goals where the total can only become lower. Should work fine with odometer resets.)

Set up a goal that will accept the meta values as an odometer goal. I use the rate of 0.1, just for it to be non-zero, but the rate is not important here, since the pressure will be not from this goal's rate but from the rates of all the goals that are taken into account.

The datapoint that is posted takes into account 10 goals with closest-to-the-road latest datapoints. Think of it as a score on the scale of 0 to 100, where 0 is "at least 10 goals are in the red today" and 100 as "all goals are at least 10 days ahead of the plan". In my experience, it make little sense to actually strive to achieve the 90+ range, as then the freedom to work more one this or that goal disappears. Instead, it is the journey that is the destination :)

**Add your INFO in the "settings.ini" file.**

After the last cell, you cap see a recap confirming that the datapoint has been posted and a list of all your goals, in which those that you can work on that contribute to the meta goal are highlighted with bold font.

ChatGPT's recap of what it helped me do:

We developed a code pipeline in Python to interact with Beeminder’s API and analyze goal data, starting by defining key parameters such as USERNAME, ACCESS_TOKEN, and BASE_URL. Using this setup, we retrieved all goals data from Beeminder and created a DataFrame to display key goal attributes. We then filtered the goals based on a specific tag ("normal") and used this filtered data to extract future goal rates and dynamically added a "rates_future" column.

Next, we calculated "max_r_future", which either took the highest or lowest value from "rates_future" based on the sign of its entries, with manual input for mixed values. We then adjusted this rate based on goal units ("d", "w", "m", "y") to create a normalized "max_r_future_adj" column. We also calculated "ahead_by_d" to show how far ahead a goal is, and rounded this value down to create an integer column, "ahead_by_d_int". Finally, we summarized the data in two variables, qu_below_10 and sum_10_below_10, reflecting how many goals were close to derailing and the adjusted sum of the lowest values.

For display, we created a new DataFrame truncated_df, formatted columns to show up to two decimals, and posted the summary data to Beeminder’s API. We wrapped up by displaying selected columns in a human-readable format, creating a well-rounded tool for monitoring Beeminder goals.

In [1]:
import requests

In [2]:
import configparser

# Initialize the ConfigParser
config = configparser.ConfigParser()

# Read the .ini file
config.read('settings.ini')

# Access the variables under the [settings] section
USERNAME = config.get('settings', 'USERNAME')
ACCESS_TOKEN = config.get('settings', 'ACCESS_TOKEN')
TAG_FOR_LIST = config.get('settings', 'TAG_FOR_LIST')
METAGOAL_SLUG = config.get('settings', 'METAGOAL_SLUG')

# Retrieve SHOW_FULL and convert it to a boolean
SHOW_FULL = config.getboolean('settings', 'SHOW_FULL')

# Display parsed settings to verify (optional)
print("Parsed Settings:")
print("USERNAME:", USERNAME)
print("ACCESS_TOKEN:", ACCESS_TOKEN)
print("TAG_FOR_LIST:", TAG_FOR_LIST)
print("METAGOAL_SLUG:", METAGOAL_SLUG)
print("SHOW_FULL:", SHOW_FULL)

Parsed Settings:
USERNAME: xxxxxxxxxxx
ACCESS_TOKEN: xxxxxxxxxxxxx
TAG_FOR_LIST: normal
METAGOAL_SLUG: meta_days_ahead
SHOW_FULL: True


In [3]:
# Base URL for Beeminder API
BASE_URL = 'https://www.beeminder.com/api/v1/'

In [4]:
# Define the endpoint URL to retrieve all goals
goals_url = f"{BASE_URL}/users/{USERNAME}/goals.json"

# Function to make a GET request and retrieve all goals
def get_all_goals():
    response = requests.get(goals_url, params={"auth_token": ACCESS_TOKEN})
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve goals: {response.status_code}")
        return None

# Test the function
all_goals = get_all_goals()
all_goals  # Display the retrieved goals


Failed to retrieve goals: 401


In [5]:
import pandas as pd

# Function to convert the retrieved goals data to a DataFrame with error checking
def goals_to_dataframe(goals_data):
    # Define the columns we want to extract
    desired_columns = ['slug', 'losedate', 'fullroad', 'delta', 'runits', 'tags', 'goal_type']
    
    # Check if goals_data is not None
    if goals_data:
        # Identify missing columns
        missing_columns = [col for col in desired_columns if col not in goals_data[0]]
        if missing_columns:
            print(f"Warning: The following columns are missing from the data and will not be included: {missing_columns}")
        
        # Extract only the available specified columns
        available_columns = [col for col in desired_columns if col in goals_data[0]]
        goals_df = pd.DataFrame(goals_data)[available_columns]
        return goals_df
    else:
        print("No data available to convert to DataFrame.")
        return pd.DataFrame()  # Return an empty DataFrame if there's no data

# Populate the DataFrame with the goals data
goals_df = goals_to_dataframe(all_goals)
goals_df  # Display the DataFrame


No data available to convert to DataFrame.


""


In [6]:
# # Filter rows where 'tags' contains TAG_FOR_LIST and 'goal_type' is not 'drinker'
# condition = goals_df['tags'].apply(lambda tags: TAG_FOR_LIST in tags if tags else False) & (goals_df['goal_type'] != 'drinker')

# # Create normal_goals_df with rows that meet the condition
# normal_goals_df = goals_df[condition]

# # Create abnormal_goals_df with rows that do not meet the condition
# abnormal_goals_df = goals_df[~condition]

# # Display the filtered DataFrames
# normal_goals_df




In [7]:
# abnormal_goals_df

In [8]:
import time

# Get the current time as a Unix timestamp
current_timestamp = int(time.time())

# Display the current timestamp
current_timestamp

1731097569

In [9]:
# Function to extract future rates based on current timestamp
def get_future_rates(fullroad, current_timestamp):
    # Filter for rates where the timestamp is in the future
    return [entry[-1] for entry in fullroad if entry[0] > current_timestamp]

# Copy DataFrame to avoid SettingWithCopyWarning and apply future rates extraction
goals_df = goals_df.copy()  # Make an explicit copy
goals_df['rates_future'] = goals_df['fullroad'].apply(lambda fullroad: get_future_rates(fullroad, current_timestamp))
# Filter out rows where 'rates_future' is an empty list
goals_df = goals_df[goals_df['rates_future'].apply(lambda x: len(x) > 0)].copy()

# Display the updated DataFrame with the new 'rates_future' column
goals_df


KeyError: 'fullroad'

In [ ]:
# Function to determine max_r_future based on specified conditions
def determine_max_r_future(slug, rates_future):
    if all(rate >= 0 for rate in rates_future):
        return max(rates_future)  # All non-negative, take max
    elif all(rate <= 0 for rate in rates_future):
        return min(rates_future)  # All non-positive, take min
    else:
        # Mixed values, prompt the user for input with full message in the console
        while True:
            try:
                user_input = float(input(
                    f"The goal '{slug}' has mixed rates in 'rates_future': {rates_future}.\n"
                    f"Please enter a value to use for '{slug}': "
                ))
                return user_input
            except ValueError:
                print("Invalid input. Please enter a numeric value.")

# Apply the function to each row in normal_goals_df and add as a new column
goals_df['max_r_future'] = goals_df.apply(
    lambda row: determine_max_r_future(row['slug'], row['rates_future']), axis=1
)

# Display the updated DataFrame
goals_df


,slug,losedate,fullroad,delta,runits,tags,goal_type,rates_future,max_r_future,max_r_future_adj,ahead_by_d,ahead_by_d_int,plus_1_units,include
2,gmail_all,1731106799,"[[1506700800, 153, 0], [1507305600, 152.775430...",4.00,w,[pin],custom,"[0, 0]",0.00,0.00,0.00,0,0.00,0
3,gmail_read,1731106799,"[[1475251200, 881, 0], [1475856000, 804.012773...",2.00,d,[pin],inboxer,[0],0.00,0.00,0.00,0,0.00,0
4,wordpress_entries,1731106799,"[[1690560000, 0, 0], [1690732800, 0, 0], [1690...",-0.14,w,[normal],hustler,[3],3.00,0.43,0.00,0,0.43,1
5,meta_days_ahead,1731106799,"[[1729958400, 27, 0], [1730044800, 27.1, 0.1],...",-0.90,d,[],biker,[0.1],0.10,0.10,0.00,0,0.10,0
6,morning_setup,1731193199,"[[1694361600, 0, 0], [1695571200, 12.6, 0.9], ...",0.82,d,[pin],hustler,[0.8799999999999999],0.88,0.88,1.93,1,0.06,0
7,yoga-stretch,1731279599,"[[1452531600, -0.7142857142857143, 0], [145313...",0.43,w,[normal],custom,"[2, 3.0000000000000027]",3.00,0.43,2.00,2,0.43,1
8,work_and_study,1731538799,"[[1602518400, 0, 0], [1602864000, 16, 4], [160...",11.86,d,[],hustler,"[0, 4]",4.00,4.00,3.96,3,0.14,0
9,photos_process,1731625199,"[[1423155600, 0, 0], [1424278800, 524.22184300...",0.00,w,[normal],custom,"[0, 105]",105.00,15.00,1.00,1,15.00,1
10,photos_remaining,1731711599,"[[1423155600, 6583, 0], [1423846800, 6583, 0],...",-13.00,w,[normal],custom,"[0, -70.66480446927405, 819.6464015773905, -70]",-70.00,-10.00,2.30,2,-7.00,1
11,tbr_total,1731711599,"[[1725984000, 41, 0], [1726588800, 41, 0], [17...",0.00,w,[bookish],custom,"[0, -1]",-1.00,-0.14,1.00,1,-0.14,0


In [ ]:
# # Filter out rows where 'max_r_future' is 0
# goals_df = goals_df.loc[goals_df['max_r_future'] != 0]

# # Display the updated DataFrame
# goals_df


In [ ]:
# Function to adjust max_r_future based on runits
def adjust_max_r_future(max_r_future, runits):
    if runits == "d":
        return max_r_future
    elif runits == "w":
        return max_r_future / 7
    elif runits == "m":
        return max_r_future / 30
    elif runits == "y":
        return max_r_future / 365
    else:
        return max_r_future  # Default in case of unexpected 'runits' value

# Apply the adjustment function to create the 'max_r_future_adj' column
goals_df['max_r_future_adj'] = goals_df.apply(
    lambda row: adjust_max_r_future(row['max_r_future'], row['runits']), axis=1
)

# Display the updated DataFrame
goals_df


,slug,losedate,fullroad,delta,runits,tags,goal_type,rates_future,max_r_future,max_r_future_adj
2,gmail_all,1731106799,"[[1506700800, 153, 0], [1507305600, 152.775430...",4.00,w,[pin],custom,"[0, 0]",0.00,0.00
3,gmail_read,1731106799,"[[1475251200, 881, 0], [1475856000, 804.012773...",2.00,d,[pin],inboxer,[0],0.00,0.00
4,wordpress_entries,1731106799,"[[1690560000, 0, 0], [1690732800, 0, 0], [1690...",-0.14,w,[normal],hustler,[3],3.00,0.43
5,meta_days_ahead,1731106799,"[[1729958400, 27, 0], [1730044800, 27.1, 0.1],...",-0.90,d,[],biker,[0.1],0.10,0.10
6,morning_setup,1731193199,"[[1694361600, 0, 0], [1695571200, 12.6, 0.9], ...",0.82,d,[pin],hustler,[0.8799999999999999],0.88,0.88
7,yoga-stretch,1731279599,"[[1452531600, -0.7142857142857143, 0], [145313...",0.43,w,[normal],custom,"[2, 3.0000000000000027]",3.00,0.43
8,work_and_study,1731538799,"[[1602518400, 0, 0], [1602864000, 16, 4], [160...",11.86,d,[],hustler,"[0, 4]",4.00,4.00
9,photos_process,1731625199,"[[1423155600, 0, 0], [1424278800, 524.22184300...",0.00,w,[normal],custom,"[0, 105]",105.00,15.00
10,photos_remaining,1731711599,"[[1423155600, 6583, 0], [1423846800, 6583, 0],...",-13.00,w,[normal],custom,"[0, -70.66480446927405, 819.6464015773905, -70]",-70.00,-10.00
11,tbr_total,1731711599,"[[1725984000, 41, 0], [1726588800, 41, 0], [17...",0.00,w,[bookish],custom,"[0, -1]",-1.00,-0.14


In [ ]:
# Function to calculate 'ahead_by_d' based on delta and max_r_future_adj
def calculate_ahead_by_d(delta, max_r_future_adj, losedate, current_timestamp, goal_type):
    # Check if delta and max_r_future_adj have opposite signs
    if (delta < 0 < max_r_future_adj) or (delta > 0 > max_r_future_adj):
        if goal_type == "drinker":
            return abs(delta / max_r_future_adj) + 1   
        else:    
            return 0  # Opposite signs, set to 0
    elif max_r_future_adj == 0:
        # Check if losedate is within 24 hours of current_timestamp
        if 0 <= losedate - current_timestamp <= 86400:  # 86400 seconds = 24 hours
            return 0
        else:
            return 100
    else:
        if goal_type == "drinker":
            return 0  
        else:
            return delta / max_r_future_adj + 1  # Same sign, calculate the value

# Apply the function to each row to create the 'ahead_by_d' column
goals_df['ahead_by_d'] = goals_df.apply(
    lambda row: calculate_ahead_by_d(row['delta'], row['max_r_future_adj'], row['losedate'], current_timestamp, row['goal_type']), axis=1
)

# Display the updated DataFrame
goals_df

,slug,losedate,fullroad,delta,runits,tags,goal_type,rates_future,max_r_future,max_r_future_adj,ahead_by_d
2,gmail_all,1731106799,"[[1506700800, 153, 0], [1507305600, 152.775430...",4.00,w,[pin],custom,"[0, 0]",0.00,0.00,0.00
3,gmail_read,1731106799,"[[1475251200, 881, 0], [1475856000, 804.012773...",2.00,d,[pin],inboxer,[0],0.00,0.00,0.00
4,wordpress_entries,1731106799,"[[1690560000, 0, 0], [1690732800, 0, 0], [1690...",-0.14,w,[normal],hustler,[3],3.00,0.43,0.00
5,meta_days_ahead,1731106799,"[[1729958400, 27, 0], [1730044800, 27.1, 0.1],...",-0.90,d,[],biker,[0.1],0.10,0.10,0.00
6,morning_setup,1731193199,"[[1694361600, 0, 0], [1695571200, 12.6, 0.9], ...",0.82,d,[pin],hustler,[0.8799999999999999],0.88,0.88,1.93
7,yoga-stretch,1731279599,"[[1452531600, -0.7142857142857143, 0], [145313...",0.43,w,[normal],custom,"[2, 3.0000000000000027]",3.00,0.43,2.00
8,work_and_study,1731538799,"[[1602518400, 0, 0], [1602864000, 16, 4], [160...",11.86,d,[],hustler,"[0, 4]",4.00,4.00,3.96
9,photos_process,1731625199,"[[1423155600, 0, 0], [1424278800, 524.22184300...",0.00,w,[normal],custom,"[0, 105]",105.00,15.00,1.00
10,photos_remaining,1731711599,"[[1423155600, 6583, 0], [1423846800, 6583, 0],...",-13.00,w,[normal],custom,"[0, -70.66480446927405, 819.6464015773905, -70]",-70.00,-10.00,2.30
11,tbr_total,1731711599,"[[1725984000, 41, 0], [1726588800, 41, 0], [17...",0.00,w,[bookish],custom,"[0, -1]",-1.00,-0.14,1.00


In [ ]:
import math

# Define a function to round down unless the value is very close to the next integer
def beeminder_floor(value):
    # Check if the value is very close to the next integer, using a small tolerance
    if math.isclose(value, math.ceil(value), abs_tol=1e-9):
        return math.ceil(value)
    else:
        return math.floor(value)

# Apply the custom rounding function to create 'ahead_by_d_int'
goals_df['ahead_by_d_int'] = goals_df['ahead_by_d'].apply(beeminder_floor)

# Display the updated DataFrame
goals_df


,slug,losedate,fullroad,delta,runits,tags,goal_type,rates_future,max_r_future,max_r_future_adj,ahead_by_d,ahead_by_d_int
2,gmail_all,1731106799,"[[1506700800, 153, 0], [1507305600, 152.775430...",4.00,w,[pin],custom,"[0, 0]",0.00,0.00,0.00,0
3,gmail_read,1731106799,"[[1475251200, 881, 0], [1475856000, 804.012773...",2.00,d,[pin],inboxer,[0],0.00,0.00,0.00,0
4,wordpress_entries,1731106799,"[[1690560000, 0, 0], [1690732800, 0, 0], [1690...",-0.14,w,[normal],hustler,[3],3.00,0.43,0.00,0
5,meta_days_ahead,1731106799,"[[1729958400, 27, 0], [1730044800, 27.1, 0.1],...",-0.90,d,[],biker,[0.1],0.10,0.10,0.00,0
6,morning_setup,1731193199,"[[1694361600, 0, 0], [1695571200, 12.6, 0.9], ...",0.82,d,[pin],hustler,[0.8799999999999999],0.88,0.88,1.93,1
7,yoga-stretch,1731279599,"[[1452531600, -0.7142857142857143, 0], [145313...",0.43,w,[normal],custom,"[2, 3.0000000000000027]",3.00,0.43,2.00,2
8,work_and_study,1731538799,"[[1602518400, 0, 0], [1602864000, 16, 4], [160...",11.86,d,[],hustler,"[0, 4]",4.00,4.00,3.96,3
9,photos_process,1731625199,"[[1423155600, 0, 0], [1424278800, 524.22184300...",0.00,w,[normal],custom,"[0, 105]",105.00,15.00,1.00,1
10,photos_remaining,1731711599,"[[1423155600, 6583, 0], [1423846800, 6583, 0],...",-13.00,w,[normal],custom,"[0, -70.66480446927405, 819.6464015773905, -70]",-70.00,-10.00,2.30,2
11,tbr_total,1731711599,"[[1725984000, 41, 0], [1726588800, 41, 0], [17...",0.00,w,[bookish],custom,"[0, -1]",-1.00,-0.14,1.00,1


In [ ]:
# Calculate "plus_1_units" and add it to the DataFrame
goals_df['plus_1_units'] = (goals_df['ahead_by_d_int'] + 1 - goals_df['ahead_by_d']) * goals_df['max_r_future_adj']

# Display the updated DataFrame
goals_df


,slug,losedate,fullroad,delta,runits,tags,goal_type,rates_future,max_r_future,max_r_future_adj,ahead_by_d,ahead_by_d_int,plus_1_units
2,gmail_all,1731106799,"[[1506700800, 153, 0], [1507305600, 152.775430...",4.00,w,[pin],custom,"[0, 0]",0.00,0.00,0.00,0,0.00
3,gmail_read,1731106799,"[[1475251200, 881, 0], [1475856000, 804.012773...",2.00,d,[pin],inboxer,[0],0.00,0.00,0.00,0,0.00
4,wordpress_entries,1731106799,"[[1690560000, 0, 0], [1690732800, 0, 0], [1690...",-0.14,w,[normal],hustler,[3],3.00,0.43,0.00,0,0.43
5,meta_days_ahead,1731106799,"[[1729958400, 27, 0], [1730044800, 27.1, 0.1],...",-0.90,d,[],biker,[0.1],0.10,0.10,0.00,0,0.10
6,morning_setup,1731193199,"[[1694361600, 0, 0], [1695571200, 12.6, 0.9], ...",0.82,d,[pin],hustler,[0.8799999999999999],0.88,0.88,1.93,1,0.06
7,yoga-stretch,1731279599,"[[1452531600, -0.7142857142857143, 0], [145313...",0.43,w,[normal],custom,"[2, 3.0000000000000027]",3.00,0.43,2.00,2,0.43
8,work_and_study,1731538799,"[[1602518400, 0, 0], [1602864000, 16, 4], [160...",11.86,d,[],hustler,"[0, 4]",4.00,4.00,3.96,3,0.14
9,photos_process,1731625199,"[[1423155600, 0, 0], [1424278800, 524.22184300...",0.00,w,[normal],custom,"[0, 105]",105.00,15.00,1.00,1,15.00
10,photos_remaining,1731711599,"[[1423155600, 6583, 0], [1423846800, 6583, 0],...",-13.00,w,[normal],custom,"[0, -70.66480446927405, 819.6464015773905, -70]",-70.00,-10.00,2.30,2,-7.00
11,tbr_total,1731711599,"[[1725984000, 41, 0], [1726588800, 41, 0], [17...",0.00,w,[bookish],custom,"[0, -1]",-1.00,-0.14,1.00,1,-0.14


In [ ]:

# Adjust the display format for floats to show up to 2 decimal places
pd.options.display.float_format = '{:.2f}'.format

# Display the DataFrame again to apply the new format
goals_df

,slug,losedate,fullroad,delta,runits,tags,goal_type,rates_future,max_r_future,max_r_future_adj,ahead_by_d,ahead_by_d_int,plus_1_units
2,gmail_all,1731106799,"[[1506700800, 153, 0], [1507305600, 152.775430...",4.00,w,[pin],custom,"[0, 0]",0.00,0.00,0.00,0,0.00
3,gmail_read,1731106799,"[[1475251200, 881, 0], [1475856000, 804.012773...",2.00,d,[pin],inboxer,[0],0.00,0.00,0.00,0,0.00
4,wordpress_entries,1731106799,"[[1690560000, 0, 0], [1690732800, 0, 0], [1690...",-0.14,w,[normal],hustler,[3],3.00,0.43,0.00,0,0.43
5,meta_days_ahead,1731106799,"[[1729958400, 27, 0], [1730044800, 27.1, 0.1],...",-0.90,d,[],biker,[0.1],0.10,0.10,0.00,0,0.10
6,morning_setup,1731193199,"[[1694361600, 0, 0], [1695571200, 12.6, 0.9], ...",0.82,d,[pin],hustler,[0.8799999999999999],0.88,0.88,1.93,1,0.06
7,yoga-stretch,1731279599,"[[1452531600, -0.7142857142857143, 0], [145313...",0.43,w,[normal],custom,"[2, 3.0000000000000027]",3.00,0.43,2.00,2,0.43
8,work_and_study,1731538799,"[[1602518400, 0, 0], [1602864000, 16, 4], [160...",11.86,d,[],hustler,"[0, 4]",4.00,4.00,3.96,3,0.14
9,photos_process,1731625199,"[[1423155600, 0, 0], [1424278800, 524.22184300...",0.00,w,[normal],custom,"[0, 105]",105.00,15.00,1.00,1,15.00
10,photos_remaining,1731711599,"[[1423155600, 6583, 0], [1423846800, 6583, 0],...",-13.00,w,[normal],custom,"[0, -70.66480446927405, 819.6464015773905, -70]",-70.00,-10.00,2.30,2,-7.00
11,tbr_total,1731711599,"[[1725984000, 41, 0], [1726588800, 41, 0], [17...",0.00,w,[bookish],custom,"[0, -1]",-1.00,-0.14,1.00,1,-0.14


In [ ]:
# Create the "include" column based on the presence of TAG_FOR_LIST in "tags"
goals_df['include'] = goals_df['tags'].apply(lambda tags: 1 if TAG_FOR_LIST in tags else 0)

# Display the updated DataFrame
goals_df

,slug,losedate,fullroad,delta,runits,tags,goal_type,rates_future,max_r_future,max_r_future_adj,ahead_by_d,ahead_by_d_int,plus_1_units,include
2,gmail_all,1731106799,"[[1506700800, 153, 0], [1507305600, 152.775430...",4.00,w,[pin],custom,"[0, 0]",0.00,0.00,0.00,0,0.00,0
3,gmail_read,1731106799,"[[1475251200, 881, 0], [1475856000, 804.012773...",2.00,d,[pin],inboxer,[0],0.00,0.00,0.00,0,0.00,0
4,wordpress_entries,1731106799,"[[1690560000, 0, 0], [1690732800, 0, 0], [1690...",-0.14,w,[normal],hustler,[3],3.00,0.43,0.00,0,0.43,1
5,meta_days_ahead,1731106799,"[[1729958400, 27, 0], [1730044800, 27.1, 0.1],...",-0.90,d,[],biker,[0.1],0.10,0.10,0.00,0,0.10,0
6,morning_setup,1731193199,"[[1694361600, 0, 0], [1695571200, 12.6, 0.9], ...",0.82,d,[pin],hustler,[0.8799999999999999],0.88,0.88,1.93,1,0.06,0
7,yoga-stretch,1731279599,"[[1452531600, -0.7142857142857143, 0], [145313...",0.43,w,[normal],custom,"[2, 3.0000000000000027]",3.00,0.43,2.00,2,0.43,1
8,work_and_study,1731538799,"[[1602518400, 0, 0], [1602864000, 16, 4], [160...",11.86,d,[],hustler,"[0, 4]",4.00,4.00,3.96,3,0.14,0
9,photos_process,1731625199,"[[1423155600, 0, 0], [1424278800, 524.22184300...",0.00,w,[normal],custom,"[0, 105]",105.00,15.00,1.00,1,15.00,1
10,photos_remaining,1731711599,"[[1423155600, 6583, 0], [1423846800, 6583, 0],...",-13.00,w,[normal],custom,"[0, -70.66480446927405, 819.6464015773905, -70]",-70.00,-10.00,2.30,2,-7.00,1
11,tbr_total,1731711599,"[[1725984000, 41, 0], [1726588800, 41, 0], [17...",0.00,w,[bookish],custom,"[0, -1]",-1.00,-0.14,1.00,1,-0.14,0


In [ ]:
# Filter rows where 'include' is 1
filtered_df = goals_df[goals_df['include'] == 1]

# Count the number of rows in the filtered DataFrame where 'ahead_by_d_int' is less than 10
qu_below_10 = (filtered_df['ahead_by_d_int'] < 10).sum()

# Get the 10 lowest values in 'ahead_by_d_int' from the filtered DataFrame and apply the min condition for summing
sum_10_below_10 = filtered_df['ahead_by_d_int'].nsmallest(10).apply(lambda x: min(x, 10)).sum()

# Display the results
qu_below_10, sum_10_below_10


(8, 54)

In [ ]:
import requests
from datetime import datetime

# Function to post a datapoint to Beeminder
def post_to_beeminder(goal_slug, value, comment):
    url = f"{BASE_URL}/users/{USERNAME}/goals/{goal_slug}/datapoints.json"
    data = {
        "auth_token": ACCESS_TOKEN,
        "value": value,
        "comment": comment
    }
    
    # Make the POST request
    response = requests.post(url, data=data)
    
    # Check if the request was successful
    if response.status_code == 200:
        print("Datapoint added successfully!")
    else:
        print(f"Failed to add datapoint: {response.status_code} - {response.json()}")

# Define the goal slug and prepare the values for posting
goal_slug = METAGOAL_SLUG
value = sum_10_below_10
current_time = datetime.now().strftime("%Y-%m-%d %H:%M")
comment = f"{qu_below_10} goals <10. Upd at {current_time}"

# Post to Beeminder
post_to_beeminder(goal_slug, value, comment)


Datapoint added successfully!


In [ ]:
from datetime import timedelta

# Function to calculate "derail_in" as "in D days HH:MM"
def calculate_derail_in(losedate, current_timestamp):
    # Calculate the time difference between losedate and the current time
    time_difference = losedate - current_timestamp
    if time_difference <= 0:
        return "Already derailed"  # Handle past or current losedate
    # Convert time difference to days, hours, and minutes
    delta = timedelta(seconds=time_difference)
    days = delta.days
    hours, remainder = divmod(delta.seconds, 3600)
    minutes = remainder // 60
    return f"in {days} d {hours:02}:{minutes:02}"

# Create a truncated copy of the DataFrame
truncated_df = goals_df.drop(columns=['fullroad', 'runits', 'tags', 'rates_future', 'max_r_future', 'goal_type']).copy()

# Calculate "derail_in" based on "losedate" and the current timestamp, then add it to the DataFrame
truncated_df['derail_in'] = truncated_df['losedate'].apply(lambda losedate: calculate_derail_in(losedate, current_timestamp))

# Drop the "losedate" column as requested
truncated_df = truncated_df.drop(columns=['losedate'])

# Display the truncated DataFrame
truncated_df


,slug,delta,max_r_future_adj,ahead_by_d,ahead_by_d_int,plus_1_units,include,derail_in
2,gmail_all,4.00,0.00,0.00,0,0.00,0,in 0 d 02:41
3,gmail_read,2.00,0.00,0.00,0,0.00,0,in 0 d 02:41
4,wordpress_entries,-0.14,0.43,0.00,0,0.43,1,in 0 d 02:41
5,meta_days_ahead,-0.90,0.10,0.00,0,0.10,0,in 0 d 02:41
6,morning_setup,0.82,0.88,1.93,1,0.06,0,in 1 d 02:41
7,yoga-stretch,0.43,0.43,2.00,2,0.43,1,in 2 d 02:41
8,work_and_study,11.86,4.00,3.96,3,0.14,0,in 5 d 02:41
9,photos_process,0.00,15.00,1.00,1,15.00,1,in 6 d 02:41
10,photos_remaining,-13.00,-10.00,2.30,2,-7.00,1,in 7 d 02:41
11,tbr_total,0.00,-0.14,1.00,1,-0.14,0,in 7 d 02:41


In [ ]:
# # Create a truncated copy of the abnormal_goals_df, dropping columns that may not exist
# columns_to_drop = ['fullroad', 'runits', 'tags', 'rates_future', 'max_r_future', 'goal_type']
# available_columns_to_drop = [col for col in columns_to_drop if col in abnormal_goals_df.columns]
# abnormal_truncated_df = abnormal_goals_df.drop(columns=available_columns_to_drop).copy()

# # Calculate "derail_in" for abnormal_truncated_df if "losedate" exists in the DataFrame
# if 'losedate' in abnormal_truncated_df.columns:
#     abnormal_truncated_df['derail_in'] = abnormal_truncated_df['losedate'].apply(lambda losedate: calculate_derail_in(losedate, current_timestamp))
#     # Drop the "losedate" column after calculation
#     abnormal_truncated_df = abnormal_truncated_df.drop(columns=['losedate'])

# # Display the truncated DataFrame for abnormal goals
# abnormal_truncated_df

In [ ]:
# import re

# # Function to extract the day integer from the "derail_in" string, defaulting to 100 if not found
# def extract_days(derail_in):
#     match = re.search(r'in (\d+) d', derail_in)
#     return int(match.group(1)) if match else 100  # Return the integer or 100 if not found

# # Apply the function to create the "der_d" column
# abnormal_truncated_df['der_d'] = abnormal_truncated_df['derail_in'].apply(extract_days)

# # Display the updated DataFrame
# abnormal_truncated_df

In [ ]:
# Function to format floats to 2 decimal places or as integers if they are close enough
def format_float(value):
    # Check if the value is already a string to avoid re-formatting
    if isinstance(value, str):
        return value
    
    # Check if the value is within 0.01 of its nearest integer
    if abs(value - round(value)) < 0.01:
        return int(round(value))  # Display as an integer if close enough
    else:
        return f"{value:.2f}"  # Otherwise, display with 2 decimal places

# Apply the formatting function to each specified column
truncated_df['delta'] = truncated_df['delta'].apply(format_float)
truncated_df['max_r_future_adj'] = truncated_df['max_r_future_adj'].apply(format_float)
truncated_df['ahead_by_d'] = truncated_df['ahead_by_d'].apply(format_float)
truncated_df['plus_1_units'] = truncated_df['plus_1_units'].apply(format_float)

# Display the nicely formatted DataFrame
truncated_df


,slug,delta,max_r_future_adj,ahead_by_d,ahead_by_d_int,plus_1_units,include,derail_in
2,gmail_all,4,0,0,0,0,0,in 0 d 02:41
3,gmail_read,2,0,0,0,0,0,in 0 d 02:41
4,wordpress_entries,-0.14,0.43,0,0,0.43,1,in 0 d 02:41
5,meta_days_ahead,-0.90,0.10,0,0,0.10,0,in 0 d 02:41
6,morning_setup,0.82,0.88,1.93,1,0.06,0,in 1 d 02:41
7,yoga-stretch,0.43,0.43,2,2,0.43,1,in 2 d 02:41
8,work_and_study,11.86,4,3.96,3,0.14,0,in 5 d 02:41
9,photos_process,0,15,1,1,15,1,in 6 d 02:41
10,photos_remaining,-13,-10,2.30,2,-7,1,in 7 d 02:41
11,tbr_total,0,-0.14,1,1,-0.14,0,in 7 d 02:41


In [ ]:
import time

# Wait for 4 seconds
time.sleep(4)

In [ ]:
# Function to get specific attributes from a Beeminder goal
def get_goal_attributes(goal_slug, attributes):
    url = f"{BASE_URL}/users/{USERNAME}/goals/{goal_slug}.json"
    response = requests.get(url, params={"auth_token": ACCESS_TOKEN})
    
    if response.status_code == 200:
        goal_data = response.json()
        # Extract the specified attributes
        return {attr: goal_data.get(attr) for attr in attributes}
    else:
        print(f"Failed to retrieve goal data: {response.status_code}")
        return None

# Retrieve "curval" and "limsum" for the specified goal
goal_slug = METAGOAL_SLUG
attributes = ["curval", "limsum"]
goal_info = get_goal_attributes(goal_slug, attributes)

# Print the results
if goal_info:
    print("curval:", goal_info["curval"])
    print("limsum:", goal_info["limsum"])

print (comment)

from IPython.display import display

# Define a function to apply row colors based on conditions
def highlight_based_on_column(row, column_name, bold=False):
    value = row[column_name]
    
    # Define color based on the value
    if value == 0:
        color = 'lightcoral'      # Red for 0
    elif value == 1:
        color = 'peachpuff'       # Orange for 1
    elif value == 2:
        color = 'palegoldenrod'   # Yellow for 2
    elif 3 <= value <= 9:
        color = 'lightgreen'      # Light green for 3-9
    else:
        color = 'mediumaquamarine'  # Green for 10 or more
    
    # Apply the color to the entire row
    styles = [f'background-color: {color}'] * len(row)
    
    # Apply bold font if SHOW_FULL is True and "include" is 1
    if bold and row['include'] == 1:
        styles = [style + '; font-weight: bold' for style in styles]
    
    return styles

# Check if SHOW_FULL is True or False
if SHOW_FULL:
    # Apply the styling function with bold option
    styled_truncated_df = truncated_df.style.apply(lambda row: highlight_based_on_column(row, 'ahead_by_d_int', bold=True), axis=1)
else:
    # Filter to include only rows where 'include' is 1, without bold styling
    truncated_df = truncated_df[truncated_df['include'] == 1]
    styled_truncated_df = truncated_df.style.apply(lambda row: highlight_based_on_column(row, 'ahead_by_d_int', bold=False), axis=1)

# Display the styled DataFrame
display(styled_truncated_df)


curval: 54
limsum: +1 in 0 days (55)
8 goals <10. Upd at 2024-11-08 21:18


,slug,delta,max_r_future_adj,ahead_by_d,ahead_by_d_int,plus_1_units,include,derail_in
2,gmail_all,4,0,0,0,0,0,in 0 d 02:41
3,gmail_read,2,0,0,0,0,0,in 0 d 02:41
4,wordpress_entries,-0.14,0.43,0,0,0.43,1,in 0 d 02:41
5,meta_days_ahead,-0.90,0.10,0,0,0.10,0,in 0 d 02:41
6,morning_setup,0.82,0.88,1.93,1,0.06,0,in 1 d 02:41
7,yoga-stretch,0.43,0.43,2,2,0.43,1,in 2 d 02:41
8,work_and_study,11.86,4,3.96,3,0.14,0,in 5 d 02:41
9,photos_process,0,15,1,1,15,1,in 6 d 02:41
10,photos_remaining,-13,-10,2.30,2,-7,1,in 7 d 02:41
11,tbr_total,0,-0.14,1,1,-0.14,0,in 7 d 02:41
